In [1]:
from pyspark.sql.types import *
from datetime import datetime

INPUT_DATA = '../data/sf-bay-area-bike-share/'

In [2]:
!head {INPUT_DATA}status.csv

station_id,bikes_available,docks_available,time
2,2,25,2013/08/29 12:06:01
2,2,25,2013/08/29 12:07:01
2,2,25,2013/08/29 12:08:01
2,2,25,2013/08/29 12:09:01
2,2,25,2013/08/29 12:10:01
2,2,25,2013/08/29 12:11:01
2,2,25,2013/08/29 12:12:01
2,2,25,2013/08/29 12:13:01
2,2,25,2013/08/29 12:15:01


In [3]:
!head {INPUT_DATA}station.csv

id,name,lat,long,dock_count,city,installation_date
2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013
3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013
6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013
7,Paseo de San Antonio,37.333798,-121.88694299999999,15,San Jose,8/7/2013
8,San Salvador at 1st,37.330165,-121.88583100000001,15,San Jose,8/5/2013
9,Japantown,37.348742,-121.89471499999999,15,San Jose,8/5/2013
10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013


In [4]:
bike_status = sc.textFile(INPUT_DATA + 'status.csv').cache()
header = bike_status.first()
bike_status = bike_status.filter(lambda x: x != header).map(lambda x: x.split(',')).cache()
bike_status.take(3)

[[u'2', u'2', u'25', u'2013/08/29 12:06:01'],
 [u'2', u'2', u'25', u'2013/08/29 12:07:01'],
 [u'2', u'2', u'25', u'2013/08/29 12:08:01']]

In [5]:
bike_status.count()

71984434

In [6]:
test_ts = bike_status.take(1)[-1][-1]
dt = datetime.strptime(test_ts, "%Y/%m/%d %H:%M:%S")

In [18]:
def toIntSafe(num):
    try:
        return int(num)
    except ValueError:
        return None
    
def toFloatSafe(num):
    try:
        return float(num)
    except ValueError:
        return None    
    
def toTimeStampSafe(data):
    try:
        return datetime.strptime(data, "%Y/%m/%d %H:%M:%S") 
    except ValueError:
        return None
    
def convertData(data):
    return (toIntSafe(data[0]),
            toIntSafe(data[1]),
            toIntSafe(data[2]),
            toTimeStampSafe(data[3]))

def preprocess_data(data):
    try:
        return (data[0], (data[2]*1.0)/(data[1] + data[2]), data[3].year,
            data[3].month, data[3].day, data[3].hour, data[3].minute, data[3].isoweekday())
    except (AttributeError, ValueError, ZeroDivisionError):
        return None

bike_status_processed = bike_status.map(lambda x:
                                        convertData(x)).map(lambda x:preprocess_data(x)).filter(lambda x: x != None).cache()

In [19]:
bike_status_processed.count()

16994602

In [20]:
bike_status_processed.take(3)

[(2, 0.9259259259259259, 2013, 8, 29, 12, 6, 4),
 (2, 0.9259259259259259, 2013, 8, 29, 12, 7, 4),
 (2, 0.9259259259259259, 2013, 8, 29, 12, 8, 4)]

In [21]:
bike_status_schema = StructType([StructField('station_id', IntegerType(),False),
                                StructField('bikes_utilised_percentage', FloatType(),False),
                                StructField('year', IntegerType(), False),
                                StructField('month', IntegerType(), False),
                                StructField('day', IntegerType(), False),
                                StructField('hour', IntegerType(), False),
                                StructField('minute', IntegerType(), False),
                                StructField('day_of_week', IntegerType(), False)])

In [22]:
bike_status_df = sqlContext.createDataFrame(bike_status_processed, bike_status_schema).cache()
bike_status_df.show(5)

+----------+-------------------------+----+-----+---+----+------+-----------+
|station_id|bikes_utilised_percentage|year|month|day|hour|minute|day_of_week|
+----------+-------------------------+----+-----+---+----+------+-----------+
|         2|                0.9259259|2013|    8| 29|  12|     6|          4|
|         2|                0.9259259|2013|    8| 29|  12|     7|          4|
|         2|                0.9259259|2013|    8| 29|  12|     8|          4|
|         2|                0.9259259|2013|    8| 29|  12|     9|          4|
|         2|                0.9259259|2013|    8| 29|  12|    10|          4|
+----------+-------------------------+----+-----+---+----+------+-----------+
only showing top 5 rows



In [23]:
bike_status_df.count()

16994602

In [24]:
bike_status_df.printSchema()

root
 |-- station_id: integer (nullable = false)
 |-- bikes_utilised_percentage: float (nullable = false)
 |-- year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- day: integer (nullable = false)
 |-- hour: integer (nullable = false)
 |-- minute: integer (nullable = false)
 |-- day_of_week: integer (nullable = false)



In [25]:
from pyspark.sql import functions as F

bike_status_period = bike_status_df.withColumn('day_part',
                                               F.when((bike_status_df.hour > 20) | (bike_status_df.hour < 6), 'night').\
                                               when((bike_status_df.hour >= 6) | (bike_status_df.hour < 12), 'morning').\
                                               when((bike_status_df.hour >= 12) | (bike_status_df.hour < 16), 'afternoon').\
                                               otherwise('evening')).cache()

bike_status_period.show(5)

+----------+-------------------------+----+-----+---+----+------+-----------+--------+
|station_id|bikes_utilised_percentage|year|month|day|hour|minute|day_of_week|day_part|
+----------+-------------------------+----+-----+---+----+------+-----------+--------+
|         2|                0.9259259|2013|    8| 29|  12|     6|          4| morning|
|         2|                0.9259259|2013|    8| 29|  12|     7|          4| morning|
|         2|                0.9259259|2013|    8| 29|  12|     8|          4| morning|
|         2|                0.9259259|2013|    8| 29|  12|     9|          4| morning|
|         2|                0.9259259|2013|    8| 29|  12|    10|          4| morning|
+----------+-------------------------+----+-----+---+----+------+-----------+--------+
only showing top 5 rows



In [26]:
bike_status_period.count()

16994602

In [27]:
daily_avg = bike_status_period.select('station_id', 'day_of_week', 'day_part', 'bikes_utilised_percentage').\
                                   groupBy('station_id', 'day_of_week', 'day_part').mean('bikes_utilised_percentage')
    
daily_avg.show(5)

+----------+-----------+--------+------------------------------+
|station_id|day_of_week|day_part|avg(bikes_utilised_percentage)|
+----------+-----------+--------+------------------------------+
|         3|          4| morning|             0.476191698166655|
|         5|          6| morning|            0.5208970950011149|
|        16|          6|   night|           0.44569035081145614|
|        22|          7|   night|            0.5165870176005278|
|        30|          1|   night|           0.44265937086872764|
+----------+-----------+--------+------------------------------+
only showing top 5 rows



In [29]:
daily_avg.cache()
daily_avg.count()

966

In [30]:
daily_avg.select('station_id').distinct().count()

69